# Tess

## Install and import things
Installations only required when the notebook server restarts (not the kernel).

In [ ]:
!pip install fsspec==0.8.4
!pip install 'git+git://github.com/tjcrone/gdrivefs.git@540a55fae06cbed4f7d9f8d540ac29fcb640c38a'

In [1]:
import xarray as xr
import pandas as pd
import gdrivefs
import cftime

## Read dataset and rename dims
Here I'm working with just sp.cdf.

In [50]:
ds = xr.open_dataset('sp.cdf', decode_times=False)
ds = ds.rename({'T': 'time', 'X': 'lat', 'Y': 'lon'})
ds.attrs['units'] = 'mbar'
ds

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 181, time: 488)
Coordinates:
  * time     (time) float32 0.5 1.5 2.5 3.5 4.5 ... 484.5 485.5 486.5 487.5
  * lat      (lat) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lon      (lon) float32 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
Data variables:
    sp       (time, lon, lat) float32 ...
Attributes:
    units:    mbar

## Convert time to datetime64
Converting to proper datetime64s will allow us to use the datetime functionality of Xarray.

In [51]:
ds['time'] = cftime.num2date(ds['time'], 'months since 1979-01-01', '360_day')
ds['time'] = ds.indexes['time'].to_datetimeindex()
ds

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  


<xarray.Dataset>
Dimensions:  (lat: 360, lon: 181, time: 488)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-16 1979-02-16 ... 2019-08-16
  * lat      (lat) float32 0.0 1.0 2.0 3.0 4.0 ... 355.0 356.0 357.0 358.0 359.0
  * lon      (lon) float32 90.0 89.0 88.0 87.0 86.0 ... -87.0 -88.0 -89.0 -90.0
Data variables:
    sp       (time, lon, lat) float32 ...
Attributes:
    units:    mbar

## Load and chunk into Dask array
Chunking into 1-day chunks seems reasonable.

In [69]:
ds = ds.load().chunk({'time': 1})
ds

,Array,Chunk
Bytes,127.19 MB,260.64 kB
Shape,"(488, 181, 360)","(1, 181, 360)"
Count,489 Tasks,488 Chunks
Type,float32,numpy.ndarray


## Save locally as Zarr

In [53]:
filename = 'sp.zarr'

In [54]:
ds.to_zarr(filename, consolidated=True, compute=True)

#### test open
This works.

In [55]:
ds_local = xr.open_zarr(filename, consolidated=True, engine='zarr', chunks='auto')

In [56]:
ds_local

,Array,Chunk
Bytes,127.19 MB,260.64 kB
Shape,"(488, 181, 360)","(1, 181, 360)"
Count,489 Tasks,488 Chunks
Type,float32,numpy.ndarray


## Zip Zarr

In [75]:
!zip -r sp.zarr.zip sp.zarr

  adding: sp.zarr/ (stored 0%)
  adding: sp.zarr/sp/ (stored 0%)
  adding: sp.zarr/sp/387.0.0 (deflated 6%)
  adding: sp.zarr/sp/455.0.0 (deflated 6%)
  adding: sp.zarr/sp/297.0.0 (deflated 8%)
  adding: sp.zarr/sp/168.0.0 (deflated 6%)
  adding: sp.zarr/sp/60.0.0 (deflated 6%)
  adding: sp.zarr/sp/337.0.0 (deflated 6%)
  adding: sp.zarr/sp/334.0.0 (deflated 6%)
  adding: sp.zarr/sp/87.0.0 (deflated 6%)
  adding: sp.zarr/sp/281.0.0 (deflated 6%)
  adding: sp.zarr/sp/95.0.0 (deflated 6%)
  adding: sp.zarr/sp/429.0.0 (deflated 6%)
  adding: sp.zarr/sp/288.0.0 (deflated 8%)
  adding: sp.zarr/sp/312.0.0 (deflated 6%)
  adding: sp.zarr/sp/343.0.0 (deflated 6%)
  adding: sp.zarr/sp/487.0.0 (deflated 6%)
  adding: sp.zarr/sp/283.0.0 (deflated 6%)
  adding: sp.zarr/sp/365.0.0 (deflated 6%)
  adding: sp.zarr/sp/369.0.0 (deflated 6%)
  adding: sp.zarr/sp/307.0.0 (deflated 6%)
  adding: sp.zarr/sp/24.0.0 (deflated 6%)
  adding: sp.zarr/sp/356.0.0 (deflated 6%)
  adding: sp.zarr/sp/406.0.0 (deflat

## Save to Google Drive as Zarr
It looks like there is a bug in the save function of gdrivefs. Nothing in this section appears to work correctly.

In [57]:
gdfid = '1PCBDhk5f3v5PoPCY3Rdcqgy4S_Yj2kCC'

In [76]:
gdfs = gdrivefs.GoogleDriveFileSystem(root_file_id=gdfid, token='cache')

In [70]:
mapper = gdfs.get_mapper(filename)

In [ ]:
ds.to_zarr(mapper, consolidated=True, append_dim='time')

In [64]:
ds.load().to_zarr(mapper, consolidated=True)
#ds.to_zarr(mapper, consolidated=True, append_dim='time')

#### test open
I downloaded the Zarr zip file and unzipped it locally, and then dragged it in to my Google Drive and this worked.

In [73]:
ds_gdrive = xr.open_zarr(mapper, consolidated=True, chunks='auto')

In [74]:
ds_gdrive

,Array,Chunk
Bytes,127.19 MB,260.64 kB
Shape,"(488, 181, 360)","(1, 181, 360)"
Count,489 Tasks,488 Chunks
Type,float32,numpy.ndarray
